## Data Cleaning Notebook for State, District, and Subdistrict Information in Census 2011
(Please note some preliminary cleaning tasks such as unmerging the merged header cells were already done in Excel.)

In [1]:
#Please run this code if you do not already have openpyxl installed in your environment.
#pip install openpyxl

In [1]:
#Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [5]:
#Importing the Excel datasheet
#But first, defining the columns that must remain as strings to avoid a loss in analytically useful information (such as "000" in str being converted to 0 in int)
column_types = {
    'State  Code': str,
    'District Code': str,
    'Sub District Code': str
}

df_census_raw = pd.read_excel('/Users/madhavsingh/Desktop/Flame/census_data/statedist_censusinfo.xlsx',
                              dtype=column_types) #Please update path and file name as necessary.

#Checking the dataset to assess the required cleaning tasks
print(df_census_raw.info())
print(df_census_raw.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19992 entries, 0 to 19991
Data columns (total 15 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   State  Code                                            19992 non-null  object 
 1   District Code                                          19992 non-null  object 
 2   Sub District Code                                      19992 non-null  object 
 3   India/ State/ Union Territory/ District/ Sub-district  19992 non-null  object 
 4   Name                                                   19992 non-null  object 
 5   Total/
Rural/
Urban                                    19992 non-null  object 
 6   Number of villages Inhabited                           19992 non-null  int64  
 7   Number of villages Uninhabited                         19992 non-null  int64  
 8   Number of towns                               

### Cleaning Tasks Part 1

Here are all the cleaning tasks that I would need to perform at first glance:

1. Change the column names to be more readable and uniform.
    - For example, "Total/Rural/Urban" should instead be "dem_class"
2. Update and/or correct the datatype for the following columns:
    - State Code, District Code, and Subdistrict Code from int to str (already done while importing, so can be removed)
    - India categorical and Total categorical from obj to category (maybe)
3. String and category consistency check for spellings, case, special characters etc.
    - Make sure that there are only 4 categories in the India categorical column and that they have the same case and spellings across types. Remove any special characters or wide spaces.
    - Remove special characters and wide spaces from the Name column. Also, standardize for case and also ensure that state spellings are consistent (e.g. Jammu & Kashmir should not appear as both Jammu & Kashmir and also Jammu and Kashmir)
    - Make sure that there are only 3 categories in the Total/Rural/Urban column and that they have the same case and spellings across types. Remove any special characters or wide spaces.

I'll first do these, then I will list and perform cleaning actions related to missingness (e.g., in area and population), sanity and logic checks (e.g., identifier codes), calculation checks (e.g., values for numerical columns in the total category should be equal to the sum of values in the urban and rural category)


In [6]:
#Cleaning Tasks Part 1

#1.1. Renaming columns
print(df_census_raw.columns, "column count:", len(df_census_raw.columns)) #old column names and length

df_census_raw.columns = ["st_code",
                        "dist_code", 
                        "sbd_code", 
                        "ent_type", 
                        "name", 
                        "dem_class", 
                        "ivlg_count", 
                        "uvlg_count", 
                        "town_count", 
                        "hhld_count", 
                        "total_pop", 
                        "male_count", 
                        "female_count", 
                        "area_sqkm", 
                        "pop_per_sqkm"]

print("1.1. Column names have been changed:")
print(df_census_raw.columns, "column count:", len(df_census_raw.columns)) #new column names and length

Index(['State  Code', 'District Code', 'Sub District Code',
       'India/ State/ Union Territory/ District/ Sub-district', 'Name',
       'Total/\nRural/\nUrban', 'Number of villages Inhabited',
       'Number of villages Uninhabited', 'Number of towns',
       'Number of households', 'Persons', 'Males', 'Females',
       'Area (in sq. km)', 'Population per sq. km.'],
      dtype='object') column count: 15
1.1. Column names have been changed:
Index(['st_code', 'dist_code', 'sbd_code', 'ent_type', 'name', 'dem_class',
       'ivlg_count', 'uvlg_count', 'town_count', 'hhld_count', 'total_pop',
       'male_count', 'female_count', 'area_sqkm', 'pop_per_sqkm'],
      dtype='object') column count: 15


In [7]:
#1.2. Changing datatypes #(Can be removed)
print(df_census_raw.dtypes) #old datatypes

df_census_raw["st_code"] = df_census_raw["st_code"].astype(str)
df_census_raw["dist_code"] = df_census_raw["dist_code"].astype(str)
df_census_raw["sbd_code"] = df_census_raw["sbd_code"].astype(str)

df_census_raw["ent_type"] = df_census_raw["ent_type"].astype(str)
df_census_raw["name"] = df_census_raw["name"].astype(str)
df_census_raw["dem_class"] = df_census_raw["dem_class"].astype(str)

print("1.2. Datatypes have been updated")
print(df_census_raw.dtypes) #new datatypes

st_code          object
dist_code        object
sbd_code         object
ent_type         object
name             object
dem_class        object
ivlg_count        int64
uvlg_count        int64
town_count        int64
hhld_count        int64
total_pop         int64
male_count        int64
female_count      int64
area_sqkm       float64
pop_per_sqkm    float64
dtype: object
1.2. Datatypes have been updated
st_code          object
dist_code        object
sbd_code         object
ent_type         object
name             object
dem_class        object
ivlg_count        int64
uvlg_count        int64
town_count        int64
hhld_count        int64
total_pop         int64
male_count        int64
female_count      int64
area_sqkm       float64
pop_per_sqkm    float64
dtype: object


In [8]:
#1.3. String and category consistency checks
print(df_census_raw["ent_type"].unique()) #can observe inconsistent categories
print(df_census_raw["dem_class"].unique())

#Removing any characters that don't belong (except from name)
df_census_raw["st_code"] = df_census_raw["st_code"].replace(r"[^0-9]", "", regex=True) #removing anything that is not a digit from the code columns
df_census_raw["dist_code"] = df_census_raw["dist_code"].replace(r"[^0-9]", "", regex=True)
df_census_raw["sbd_code"] = df_census_raw["sbd_code"].replace(r"[^0-9]", "", regex=True)
df_census_raw["ent_type"] = df_census_raw["ent_type"].replace(r"[^a-zA-Z\-]", "", regex=True) #we want to preserve the hyphen for the "SUB-DISTRICT" category
df_census_raw["dem_class"] = df_census_raw["dem_class"].replace(r"[^a-zA-Z]", "", regex=True)

#Case standardization of categories
df_census_raw["ent_type"] = df_census_raw["ent_type"].str.upper()
df_census_raw["dem_class"] = df_census_raw["dem_class"].str.upper()

print("1.3. String values (except name) have been cleaned.")
print(df_census_raw["ent_type"].unique())
print(df_census_raw["dem_class"].unique())

['INDIA' 'STATE' 'DISTRICT' 'SUB-DISTRICT']
['Total' 'Rural' 'Urban']
1.3. String values (except name) have been cleaned.
['INDIA' 'STATE' 'DISTRICT' 'SUB-DISTRICT']
['TOTAL' 'RURAL' 'URBAN']


In [9]:
#1.3. (contd.) #Name standardization
print(df_census_raw.sample(5)) #can observe very unclean name strings

#First, we print all cell values for name column where there occur any non alphanumeric characters except for spaces to establish the baseline.
for index, row in df_census_raw.iterrows():
    if re.search(r'[^a-zA-Z0-9\s]', str(row["name"])):
        print("Old Name:", row["name"])

#As we can see, there are a lot of special characters and string inconsistences in the Name column.
#However, due to the unique nature of the names, we cannot remove everything automatically.
#We will have to think more closely. Let's define a function to do this cleaning.

def census_namecleaner_v1(text):
    if not isinstance(text, str):
        return text

    # 1. Cleaning boundary noise (but keeping dots and quotes as they may have special meaning such as in "V.K.")
    text = re.sub(r'^[ @&^*#$]+|[ @&^*#$]+$', '', text)

    # 2. Normalizing all whitespaces to a single space
    text = re.sub(r'\s+', ' ', text)

    # 3. Removing spaces around periods (to avoid discrepancies between "V.K." and "V. K.")
    text = re.sub(r'\s*\.\s*', '.', text)

    # 4. Fixing spacing around parantheses
    text = re.sub(r'\(\s+', '(', text)
    text = re.sub(r'\s+\)', ')', text)
    text = re.sub(r'(?<=[^\s])\(', r' (', text)
    # Also ensure space after closing parenthesis if text follows
    text = re.sub(r'\)(?=[a-zA-Z0-9])', r') ', text)

    # 4. Handling Case-Merging (e.g., GayaTown -> Gaya Town)
    text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)

    # 5. Standardizing Hyphens and Slashes
    text = re.sub(r'\s*-\s*', '-', text)
    text = re.sub(r'\s*/\s*', '/', text)

    return text.strip()

# Applying to the dataframe
df_census_raw['name'] = df_census_raw['name'].apply(census_namecleaner_v1)

#Now, let's check the cleaned names:
for index, row in df_census_raw.iterrows():
    if re.search(r'[^a-zA-Z0-9\s]', str(row["name"])):
        print("New Name:", row["name"])

#Finally, let us also standardize the cases of India and of state names in the name column to title case.
df_census_raw.loc[df_census_raw["ent_type"].isin(["INDIA", "STATE"]), "name"] = df_census_raw["name"].str.title()

print("1.3. (contd.) Entity names have been cleaned and standardized.")
print(df_census_raw.sample(5))
print(df_census_raw.head())

      st_code dist_code sbd_code      ent_type  \
8200       19       337    02329  SUB-DISTRICT   
8666       19       344    99999  SUB-DISTRICT   
12637      23       457    03675  SUB-DISTRICT   
13337      24       484    03880  SUB-DISTRICT   
5121       10       234    01463  SUB-DISTRICT   

                                  name dem_class  ivlg_count  uvlg_count  \
8200                      Basirhat - I     RURAL          59           1   
8666   Area not under any Sub-district     URBAN           0           0   
12637                        Paraswada     RURAL         163          10   
13337                           Kadana     URBAN           0           0   
5121                         Nasriganj     TOTAL          50           3   

       town_count  hhld_count  total_pop  male_count  female_count  \
8200            0       33417     150520       76930         73590   
8666            8      123268     564913      286679        278234   
12637           0       23138   

### Cleaning Tasks Part 2

1. Check and Handle Missingness
    - There may be two kinds of missingness: one literal and one camouflaged.
    - For example, literal missingness is just all the NA or nan values in the dataset. These only appear in the columns of "area_sqkm" and "pop_per_sqkm". 
        - I will have to decide how to handle these. Most likely, I am going to leave them as is because they are so few in number. However, if I can find relevant data for these, I will manually impute it.
    - On the other side of the coin is the camouflaged missingness, where there is an actual entry but it does not provide any analytically useful information. For example, in the "name" column, there are several entries of "Area not under any Sub-district." These also correspond with the "99999" subdistrict code. Now, although these entries are not actually missing as nan values, they still do not reveal much about the entity in question.
        - To handle such entries, I will first have to see the corresponding information in all other columns. For example, if there exists information for population or area for some "Area not under any Sub-district" entry, then I may have to manually impute a new name corresponding to the state under which it appears to make the entry analytically useful. Such as "Unclassified Area Under District: [District Name] in State: [State Name]. Let's see.

2. Sanity and Logic Checks
    - Make sure that (1) "st_code" is "00" ONLY for the "ent_type" == INDIA entries, (2) "dist_code" is "000" ONLY for "ent_type" is in ["INDIA", "STATE"] entries, and (3) "sbd_code" is "00000" ONLY for "ent_type" is in ["INDIA", "STATE", "DISTRICT"] entries. Also, make sure that each of the code is 2 digit, 3 digit, and 5 digit in length, respectively for all entries.
    - Ensure that there are no negative values in the summary statistics columns (population, area, etc.). These values should not be possible and must be converted to 0 or nan.
    - There's also a weird issue with the entries in the "pop_per_sqkm" column. Till around the 5435th index (can be accurately checked via code), the entries for population per sq km are integer numbers with no decimals. But from the 5438th entry till the 18177th entry, they are all float values with decimals. Then, from 18179th entry till the end, they are integers again. Additionally, in the excel sheet, they are all reported as integers, with the float values being rounded to the nearest integer.
        - To handle this, I will probably have to standardize the values by either (1) manually calculating population per sq km from the total_pop and area_per_sqkm columns or (2) rounding off all the float values to their nearest integer. Let's see which ends up being more robust.
    - Given the format of the dataset, there must be EXACTLY THREE INSTANCES of every unique entry in the dataset (defined and identified by a new column that stores the combination of their state code, district code, and subdistrict code; for example, the district of name "Kupwara" gets the unique identity tag of "0100100000" whereas the subdistrict of the same name "Kupwara" gets the unique identity tag of "0100100001"), with each of the three instances corresponding to a different demographic classification (one instance for "TOTAL", one for "RURAL", and one for "URBAN"). Flag any entry that either has (1) more or less than three instances (e.g., "0100100000" appears more or less than three times in the dataset) or (2) has repeated entries of the same category in the "dem_class" column (e.g., hypothetically, "0100100000" appears three times in the dataset, but two of the three entries are marked as "RURAL" in the "dem_class" column).
    - While some or all of the descriptive statistics columns (e.g., ivlg_count, uvlg_count, and so on) for either the "RURAL" category or the "URBAN" category may be 0 (e.g., when there are no villages and towns classified as "URBAN" in a certain subdistrict "Khag", thus automatically rendering the ivlg, uvlg, town, hhld, and so on statistics for the urban row of that subdistrict to 0), there should be NO 0 VALUES in the descriptive statistics columns for any row where the dem_class == "TOTAL". If there are such 0 entries found in the "TOTAL" rows, they should be examined and treated as missing values.
    - If for any row there exists a non-0 and non-missing value in either "ivlg_count", "uvlg_count", or "town_count" columns, then there should be a corresponding non-0 and non-missing value in the "hhld_count" column and, by corollary, also in the "total_pop", "male_count", and "female_count" columns.

3. Calculation Checks
    - For each unique entry (signified by the same combination of codes elaborated on before), the values in all the descriptive statistics columns (from "ivlg_count" to "area_sqkm" and not "pop_per_sqkm") for instance where "dem_class" == "TOTAL" must be equal to the sum of the respective column values obtained from "dem_class" == "URBAN" and "dem_class" == "RURAL". (Unless missing, but these entries should be flagged)
    - For each row, the value in the "total_pop" column must be equal to the sum of the values in the "male_count" and "female_count" columns. (Unless missing, but these entries should be flagged)
    - (Note: Since pop_per_sqkm column will probably be manually reconstructed due to the float-int discrepancies, we do not need to check its calculations)


In [10]:
## Cleaning Tasks Part 2

#2.1.1. Checking for Literal Missingness
print(df_census_raw.isnull().sum())

#Enforcing missingness because there is no way to impute these values, maybe one can manually look up the area in sqkm of these entities as of 2011.
df_census_raw.loc[df_census_raw["area_sqkm"].isnull(), "area_sqkm"] = np.nan
df_census_raw.loc[df_census_raw["pop_per_sqkm"].isnull(), "pop_per_sqkm"] = np.nan

print("2.1.1. Literal missingness has been accounted for.")

st_code           0
dist_code         0
sbd_code          0
ent_type          0
name              0
dem_class         0
ivlg_count        0
uvlg_count        0
town_count        0
hhld_count        0
total_pop         0
male_count        0
female_count      0
area_sqkm       117
pop_per_sqkm    117
dtype: int64
2.1.1. Literal missingness has been accounted for.


In [11]:
#2.1.2. Checking for Camouflaged Missingness in "name" column
df_missing_names = df_census_raw[df_census_raw['name'].str.contains(r'^Area not', flags=re.IGNORECASE, na=False)]
print(df_missing_names.sample(5))

#Now, checking if summary statistics for the "TOTAL" Category in this missing subset are not NA.
print(df_missing_names.loc[df_missing_names["dem_class"] == "TOTAL", df_missing_names.columns[7:15]].isnull().sum())
#Since there are no NA values in the summary statistics of these entries, that means these are valid entries even if their name is not analytically useful.
#For now, we can leave their names as is since the "Area not under any Sub-district" name is recognized as a valid entry in documents and discussions. 
print("2.1.2. Camouflaged missingness has been accounted for.")

      st_code dist_code sbd_code      ent_type  \
8358       19       339    99999  SUB-DISTRICT   
8478       19       342    99999  SUB-DISTRICT   
8359       19       339    99999  SUB-DISTRICT   
7695       19       328    99999  SUB-DISTRICT   
18426      29       563    99999  SUB-DISTRICT   

                                  name dem_class  ivlg_count  uvlg_count  \
8358   Area not under any Sub-district     TOTAL           0           0   
8478   Area not under any Sub-district     TOTAL           0           0   
8359   Area not under any Sub-district     RURAL           0           0   
7695   Area not under any Sub-district     TOTAL           0           0   
18426  Area not under any Sub-district     TOTAL           0           0   

       town_count  hhld_count  total_pop  male_count  female_count  area_sqkm  \
8358            3       51260     234254      118886        115368      59.08   
8478            1     1024928    4496694     2356766       2139928     185.00   

In [12]:
#2.2.1. Identifying Code Sanity Checks

#First, I check whether each entry for the three codes is 2, 3, and 5 digits long, respectively.
for index, row in df_census_raw.iterrows():
    assert len(row["st_code"]) == 2
    assert len(row["dist_code"]) == 3
    assert len(row["sbd_code"]) == 5 

#Second, I check whether the empty codes ("00", "000", "00000") are used appropriately. For example, "00" in st_code is only meant for India, "000" in dist_code is only meant for India and [State Name] entries, and "00000" in sbd_code is only meant for India, [State Name], and [District Name] entries.

print("Check for INDIA only:", df_census_raw.loc[df_census_raw["st_code"] == "00", "ent_type"]) #This can be manually checked since it should only return three entries is correct.
print("Check for INDIA and STATE only:", df_census_raw.loc[df_census_raw["dist_code"] == "000", "ent_type"]) #This can be manually checked, but it is better to run a loop to check.
print("Check for INDIA and STATE and DISTRICT only:", df_census_raw.loc[df_census_raw["sbd_code"] == "00000", "ent_type"]) #These are WAY too many entries to check manually. So, I will have to run a for loop for this loc code to check that the "ent_type" is not in ["SUBDISTRICT"]

all_match1 = True
for value in df_census_raw.loc[df_census_raw["dist_code"] == "000", "ent_type"]:
    if value == "DISTRICT" or value == "SUBDISTRICT":
        all_match = False
        break
if all_match1:
    print("District Code Checks out")

all_match2 = True
for value in df_census_raw.loc[df_census_raw["sbd_code"] == "00000", "ent_type"]:
    if value == "SUBDISTRICT":
        all_match = False
        break
if all_match2:
    print("Subdistrict Code Checks out")

print("2.2.1. State, District, and Subdistrict Codes are correctly assigned and of the correct length.")

Check for INDIA only: 0    INDIA
1    INDIA
2    INDIA
Name: ent_type, dtype: object
Check for INDIA and STATE only: 0        INDIA
1        INDIA
2        INDIA
3        STATE
4        STATE
         ...  
19915    STATE
19916    STATE
19953    STATE
19954    STATE
19955    STATE
Name: ent_type, Length: 108, dtype: object
Check for INDIA and STATE and DISTRICT only: 0           INDIA
1           INDIA
2           INDIA
3           STATE
4           STATE
           ...   
19969    DISTRICT
19970    DISTRICT
19980    DISTRICT
19981    DISTRICT
19982    DISTRICT
Name: ent_type, Length: 2028, dtype: object
District Code Checks out
Subdistrict Code Checks out
2.2.1. State, District, and Subdistrict Codes are correctly assigned and of the correct length.


In [13]:
#2.2.2. Sanity Checking Summary Statistics Columns for Negative Values
sumstats = df_census_raw.iloc[:, 6:15]
sumstats

negative_check = (sumstats < 0).any()
print("Columns with negative values:")
print(negative_check)
negative_counts = (sumstats < 0).sum() #Print the exact count of negative values per column
print("\nCount of negative values per column:")
print(negative_counts)

#Manually verifying the negative entries and changing them to 0 or nan.
print(df_census_raw[df_census_raw["area_sqkm"] < 0]) 
df_census_raw.loc[df_census_raw["area_sqkm"] < 0, "area_sqkm"] = 0 #Changing area_sqkm to 0 because all other column entries for these rows are 0, likely indicating non-existence and not missingness.
if df_census_raw[df_census_raw["area_sqkm"] < 0].empty: #Verifying (should be empty)
    print("Negative values have been changed to 0.")

Columns with negative values:
ivlg_count      False
uvlg_count      False
town_count      False
hhld_count      False
total_pop       False
male_count      False
female_count    False
area_sqkm        True
pop_per_sqkm    False
dtype: bool

Count of negative values per column:
ivlg_count      0
uvlg_count      0
town_count      0
hhld_count      0
total_pop       0
male_count      0
female_count    0
area_sqkm       2
pop_per_sqkm    0
dtype: int64
      st_code dist_code sbd_code      ent_type        name dem_class  \
13531      24       492    03937  SUB-DISTRICT  Surat City     RURAL   
15355      28       536    00000      DISTRICT   Hyderabad     RURAL   

       ivlg_count  uvlg_count  town_count  hhld_count  total_pop  male_count  \
13531           0           0           0           0          0           0   
15355           0           0           0           0          0           0   

       female_count  area_sqkm  pop_per_sqkm  
13531             0  -0.000582           0

In [14]:
#2.2.3. Standardizing the Datatype of pop_per_sqkm Values
#First, let's check for 0/0 divisions and value/0 divisions before manually calculating the new pop_per_sqkm column
print("Number of 0/0 divisions found:", len(df_census_raw[(df_census_raw["total_pop"] == 0) & (df_census_raw["area_sqkm"] == 0)])) # 2827 entries for a 0/0 division. These probably correspond to non-existent URBAN areas within districts/subdistricts classified entirely as RURAL and vice versa.
#Since python will turn a 0/0 division into nan, I will need to use numpy for handling these cases.
if df_census_raw[(df_census_raw["total_pop"] != 0) & (df_census_raw["area_sqkm"] == 0)].empty: #Checking for infinite values.
    print("No 0/0 divisions found.") 

#Now, applying conditional division to handle 0/0, !0/0, 0/missing, and missing/0 divisions to create a new pop_per_sqkm column
#I only force a '0' result if: Population is exactly 0 AND Area is 0 AND NOT NaN
poparea_condition = (df_census_raw["total_pop"] == 0) & (df_census_raw["area_sqkm"].notna())

df_census_raw["pop_per_sqkm_new"] = np.where(
    poparea_condition, 
    0, 
    df_census_raw["total_pop"] / df_census_raw["area_sqkm"]
)

print(df_census_raw.head(5))
#Now, as we can see, the manual calculations sometimes differ markedly from (at least) the int calculations of pop_per_sqkm column.
#For now, it is better to just treat the new, manually created pop_per_sqkm_new as the correct one. I will, however, still maintain the old column for transparency, but renaming it to pop_per_sqkm_old
df_census_raw = df_census_raw.rename(columns={'pop_per_sqkm': 'pop_per_sqkm_old'})
print("pop_per_sqkm has been standardized into float values as a new column: pop_per_sqkm_new.")
df_census_raw.columns

Number of 0/0 divisions found: 2827
No 0/0 divisions found.
  st_code dist_code sbd_code ent_type             name dem_class  ivlg_count  \
0      00       000    00000    INDIA            India     TOTAL      597608   
1      00       000    00000    INDIA            India     RURAL      597608   
2      00       000    00000    INDIA            India     URBAN           0   
3      01       000    00000    STATE  Jammu & Kashmir     TOTAL        6337   
4      01       000    00000    STATE  Jammu & Kashmir     RURAL        6337   

   uvlg_count  town_count  hhld_count   total_pop  male_count  female_count  \
0       43324        7933   249501663  1210854977   623270258     587584719   
1       43324           0   168612897   833748852   427781058     405967794   
2           0        7933    80888766   377106125   195489200     181616925   
3         216         122     2119718    12541302     6640662       5900640   
4         216           0     1553433     9108060     4774477   

Index(['st_code', 'dist_code', 'sbd_code', 'ent_type', 'name', 'dem_class',
       'ivlg_count', 'uvlg_count', 'town_count', 'hhld_count', 'total_pop',
       'male_count', 'female_count', 'area_sqkm', 'pop_per_sqkm_old',
       'pop_per_sqkm_new'],
      dtype='object')

In [15]:
#2.2.4. Creating Unique Identification for Entries and Sanity Checking Instances by "dem_class"
#Let's create two new columns at the very first and second position in the dataset called:
#(a). "ent_id" by concatinating "st_code", "dist_code", "sbd_code".
#(b). "unique_id" by concatinating "st_code", "dist_code", "sbd_code", and the first letter of "dem_class".

st_code = df_census_raw["st_code"].astype(str)
dist_code = df_census_raw["dist_code"].astype(str)
sbd_code = df_census_raw["sbd_code"].astype(str)

ent_id_values = st_code + dist_code + sbd_code
unique_id_values = st_code + dist_code + sbd_code + df_census_raw["dem_class"].str[0]

if "ent_id" not in df_census_raw.columns:
    df_census_raw.insert(0, "ent_id", ent_id_values)

if "unique_id" not in df_census_raw.columns:
    df_census_raw.insert(1, "unique_id", unique_id_values)

print(df_census_raw[["ent_id", "unique_id", "name", "dem_class"]].head(10))

if len(df_census_raw["unique_id"].unique()) == len(df_census_raw):
    print("Unique ID has been successfully created.")

if len(df_census_raw["ent_id"].unique())*3 == len(df_census_raw):
    print("Ent ID has been successfully created.")

#Now, on to the sanity checks:
#Grouping by ent_id and dem_class to find the counts
#This will identify if any specific category (e.g., RURAL) is repeated for an ID
group_counts = df_census_raw.groupby(['ent_id', 'dem_class']).size().reset_index(name='category_occurrence')

#Identifying Repeats (Issue #2: e.g., two "RURAL" entries for the same ID)
repeated_categories = group_counts[group_counts['category_occurrence'] > 1]

#Identifying Count Mismatches (Issue #1: More or less than 3 instances)
# We count how many unique 'dem_class' values exist per 'ent_id'
ent_summary = df_census_raw.groupby('ent_id')['dem_class'].nunique().reset_index(name='unique_dem_count')
mismatched_counts = ent_summary[ent_summary['unique_dem_count'] != 3]

print("--- Sanity Check Results ---")

if repeated_categories.empty:
    print("✓ No duplicate demographic categories found per entry.")
else:
    print(f"⚠ FLAG: Found {len(repeated_categories)} instances of repeated categories!")
    print(repeated_categories.head())

if mismatched_counts.empty:
    print("✓ Every entry has exactly 3 unique demographic categories.")
else:
    print(f"⚠ FLAG: Found {len(mismatched_counts)} entries without exactly 3 instances!")
    print(mismatched_counts.head())

       ent_id    unique_id             name dem_class
0  0000000000  0000000000T            India     TOTAL
1  0000000000  0000000000R            India     RURAL
2  0000000000  0000000000U            India     URBAN
3  0100000000  0100000000T  Jammu & Kashmir     TOTAL
4  0100000000  0100000000R  Jammu & Kashmir     RURAL
5  0100000000  0100000000U  Jammu & Kashmir     URBAN
6  0100100000  0100100000T          Kupwara     TOTAL
7  0100100000  0100100000R          Kupwara     RURAL
8  0100100000  0100100000U          Kupwara     URBAN
9  0100100001  0100100001T          Kupwara     TOTAL
Unique ID has been successfully created.
Ent ID has been successfully created.
--- Sanity Check Results ---
✓ No duplicate demographic categories found per entry.
✓ Every entry has exactly 3 unique demographic categories.


In [16]:
#2.2.5. Sanity Checking Summary Statistics Columns for the "TOTAL" Category
# Group A: The "Existence" check (at least one must be > 0)
#Flags "TOTAL" rows where all settlement counts (villages and towns) are zero, identifying "Ghost Entries" that lack any physical administrative units.
settlement_cols = ['ivlg_count', 'uvlg_count', 'town_count']

# Group B: The "Population" and "Area" check (must be > 0)
#Flags "TOTAL" rows with zero population or households, as administrative units must contain residents to be valid census records.
poparea_cols = ['hhld_count', 'total_pop', 'male_count', 'female_count', "area_sqkm", "pop_per_sqkm_new"]

#Filtering for 'TOTAL' rows that fail either existence or population and area check (Because it should be impossible for a total entry to have 0)
print(df_census_raw[
    (df_census_raw["dem_class"] == "TOTAL") & 
    ((df_census_raw[settlement_cols] == 0).all(axis=1) | (df_census_raw[poparea_cols] == 0).any(axis=1))
])

print("Found", len(df_census_raw[(df_census_raw["dem_class"] == "TOTAL") & ((df_census_raw[settlement_cols] == 0).all(axis=1) | (df_census_raw[poparea_cols] == 0).any(axis=1))]), "'TOTAL' rows failing the logic checks.")
#Whether these entries should stay as 0 or be converted to NaN will depend on manual checking. 
#Some areas such as Connaught Place are commercial neighbourhoods, so it makes sense they do not report any village or town counts. In this case, the 0 entries make sense. 
#But for the 34 entries, this will have to be decided on a case-by-case basis.

           ent_id    unique_id st_code dist_code sbd_code      ent_type  \
1596   0709100434  0709100434T      07       091    00434  SUB-DISTRICT   
1599   0709100435  0709100435T      07       091    00435  SUB-DISTRICT   
1620   0709300440  0709300440T      07       093    00440  SUB-DISTRICT   
1626   0709400000  0709400000T      07       094    00000      DISTRICT   
1629   0709400442  0709400442T      07       094    00442  SUB-DISTRICT   
1632   0709400443  0709400443T      07       094    00443  SUB-DISTRICT   
1635   0709400444  0709400444T      07       094    00444  SUB-DISTRICT   
1638   0709500000  0709500000T      07       095    00000      DISTRICT   
1641   0709500445  0709500445T      07       095    00445  SUB-DISTRICT   
1644   0709500446  0709500446T      07       095    00446  SUB-DISTRICT   
1647   0709500447  0709500447T      07       095    00447  SUB-DISTRICT   
1656   0709600449  0709600449T      07       096    00449  SUB-DISTRICT   
14325  2751900000  275190

In [17]:
#2.2.5. Sanity Checking Missingness in Population Columns (where inhabited settlements exist but the total population is 0 or missing)

if df_census_raw[df_census_raw[['ivlg_count', 'town_count']].gt(0).any(axis=1) & df_census_raw['total_pop'].fillna(0).le(0)].empty:
    print("No entries with inhabited settlements but zero or missing population.")


No entries with inhabited settlements but zero or missing population.


In [18]:
#2.3.1. Checking Calculations of Summary Statistics Columns in the "TOTAL" Category as Summation of "RURAL" and "URBAN" statistics
#Defining all columns together.
sumstats_cols = ['ivlg_count', 'uvlg_count', 'town_count', 'hhld_count', 
            'total_pop', 'male_count', 'female_count', 'area_sqkm']

#Pivoting the data so we have 'TOTAL', 'RURAL', and 'URBAN' as separate columns for every metric
pivot = df_census_raw.pivot_table(index='ent_id', columns='dem_class', values=sumstats_cols)

#Calculating the difference: TOTAL - (RURAL + URBAN)
#If the data is consistent, the result for every cell should be 0
diff = pivot.xs('TOTAL', axis=1, level=1) - (pivot.xs('RURAL', axis=1, level=1) + pivot.xs('URBAN', axis=1, level=1))

#Flagging any entry where the difference is NOT zero in ANY column
mismatched_entries = diff[(diff != 0).any(axis=1)]

print(f"Found {len(mismatched_entries)} entries with summation inconsistencies.")
mismatched_entries

#TO BE HANDLED LATER MANUALLY. All inconsistencies are ONLY in the "area_sqkm" column. 
#Most of these inconsistencies are due to rounding errors or floating point artifacts since they are EXTREMELY small.
#The only large discrepancy is for India, probably due to the influence of the contested areas of JnK.

Found 227 entries with summation inconsistencies.


,area_sqkm,female_count,hhld_count,ivlg_count,male_count,total_pop,town_count,uvlg_count
ent_id,,,,,,,,
0000000000,8.374300e+04,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0100200008,-2.842171e-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0100200009,7.105427e-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0100500019,-2.842171e-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0100700030,-1.136868e-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
3360705734,-1.136868e-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3361405778,1.047409e-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3361405779,-2.668137e-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
#2.3.2. Checking Calculations of "total_pop" column
#Flagging any rows where the gender sum is incorrect
gender_error = df_census_raw[df_census_raw['total_pop'] != (df_census_raw['male_count'] + df_census_raw['female_count'])]

print(f"Found {len(gender_error)} rows where Male + Female does not equal Total Population.")
gender_error[['name', 'dem_class', 'total_pop', 'male_count', 'female_count']].head()

Found 0 rows where Male + Female does not equal Total Population.


,name,dem_class,total_pop,male_count,female_count


In [ ]:
df_census_clean = df_census_raw
df_census_clean.info()
#df_census_clean.to_csv('census_clean.csv', index=False) (Remove hash to export the cleaned dataset)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19992 entries, 0 to 19991
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ent_id            19992 non-null  object 
 1   unique_id         19992 non-null  object 
 2   st_code           19992 non-null  object 
 3   dist_code         19992 non-null  object 
 4   sbd_code          19992 non-null  object 
 5   ent_type          19992 non-null  object 
 6   name              19992 non-null  object 
 7   dem_class         19992 non-null  object 
 8   ivlg_count        19992 non-null  int64  
 9   uvlg_count        19992 non-null  int64  
 10  town_count        19992 non-null  int64  
 11  hhld_count        19992 non-null  int64  
 12  total_pop         19992 non-null  int64  
 13  male_count        19992 non-null  int64  
 14  female_count      19992 non-null  int64  
 15  area_sqkm         19875 non-null  float64
 16  pop_per_sqkm_old  19875 non-null  float6